# Refactoring This Mess

---
# First, get the data

In [1]:
import pandas as pd

dexseqResults = pd.read_table("/home/dwito/BonnalData/dexseq_rmats_analysis/dexseq_res/bonnal_dexseq_results.txt")

A3SS_MATS = pd.read_table("/home/dwito/BonnalData/dexseq_rmats_analysis/rmats_out/A3SS.MATS.JCEC.txt", dtype=str)
A3SS_MATS["ID"] = "A3SS_" + A3SS_MATS["ID"].astype(str)

A5SS_MATS = pd.read_table("/home/dwito/BonnalData/dexseq_rmats_analysis/rmats_out/A5SS.MATS.JCEC.txt", dtype=str)
A5SS_MATS["ID"] = "A5SS_" + A5SS_MATS["ID"].astype(str)

SE_MATS = pd.read_table("/home/dwito/BonnalData/dexseq_rmats_analysis/rmats_out/SE.MATS.JCEC.txt", dtype=str)
SE_MATS["ID"] = "SE_" + SE_MATS["ID"].astype(str)

RI_MATS = pd.read_table("/home/dwito/BonnalData/dexseq_rmats_analysis/rmats_out/RI.MATS.JCEC.txt", dtype=str)
RI_MATS["ID"] = "RI_" + RI_MATS["ID"].astype(str)

---
# Counting Exons

## Get dex_to_rmats

In [2]:
dex_to_A3SS = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.dexseq.A3SS.mapped.txt",
	dtype=str)
dex_to_A3SS = dex_to_A3SS[dex_to_A3SS["GeneID"] != "GeneID"]
dex_to_A3SS["GeneID"] = dex_to_A3SS["GeneID"].str.strip()
dex_to_A3SS["DexseqFragment"] = dex_to_A3SS["DexseqFragment"].str.strip()
dex_to_A3SS = dex_to_A3SS.sort_values(by=["GeneID", "DexseqFragment"])
dex_to_A3SS = dex_to_A3SS.reset_index(drop=True)

dex_to_A5SS = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.dexseq.A5SS.mapped.txt",
	dtype=str)
dex_to_A5SS = dex_to_A5SS[dex_to_A5SS["GeneID"] != "GeneID"]
dex_to_A5SS["GeneID"] = dex_to_A5SS["GeneID"].str.strip()
dex_to_A5SS["DexseqFragment"] = dex_to_A5SS["DexseqFragment"].str.strip()
dex_to_A5SS = dex_to_A5SS.sort_values(by=["GeneID", "DexseqFragment"])
dex_to_A5SS = dex_to_A5SS.reset_index(drop=True)

dex_to_SE = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.dexseq.SE.mapped.txt",
	dtype=str)
dex_to_SE = dex_to_SE[dex_to_SE["GeneID"] != "GeneID"]
dex_to_SE["GeneID"] = dex_to_SE["GeneID"].str.strip()
dex_to_SE["DexseqFragment"] = dex_to_SE["DexseqFragment"].str.strip()
dex_to_SE = dex_to_SE.sort_values(by=["GeneID", "DexseqFragment"])
dex_to_SE = dex_to_SE.reset_index(drop=True)

dex_to_RI = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.dexseq.RI.mapped.txt",
	dtype=str)
dex_to_RI = dex_to_RI[dex_to_RI["GeneID"] != "GeneID"]
dex_to_RI["GeneID"] = dex_to_RI["GeneID"].str.strip()
dex_to_RI["DexseqFragment"] = dex_to_RI["DexseqFragment"].str.strip()
dex_to_RI = dex_to_RI.sort_values(by=["GeneID", "DexseqFragment"])
dex_to_RI = dex_to_RI.reset_index(drop=True)

dex_to_SE_A5 = pd.merge(dex_to_SE, dex_to_A5SS, how="outer", on=["GeneID", "DexseqFragment"])
del dex_to_SE
del dex_to_A5SS
dex_to_SE_A5_A3 = pd.merge(dex_to_SE_A5, dex_to_A3SS, how="outer", on=["GeneID", "DexseqFragment"])
del dex_to_A3SS
del dex_to_SE_A5
dex_to_rmats = pd.merge(dex_to_SE_A5_A3, dex_to_RI, how="outer", on=["GeneID", "DexseqFragment"])
del dex_to_RI
del dex_to_SE_A5_A3

## Merge dex_to_rmats with DEXSeq results

In [3]:
dex_to_rmats["rMATS_ID"] = dex_to_rmats[["rMATS_ID_A3SS", "rMATS_ID_A5SS", "rMATS_ID_SE", "rMATS_ID_RI"]].stack().groupby(level=0).agg(','.join)
dex_to_rmats = dex_to_rmats.drop(columns=["rMATS_ID_A3SS", "rMATS_ID_A5SS", "rMATS_ID_SE", "rMATS_ID_RI"])

dex_to_rmats_dexRes = pd.merge(dexseqResults, dex_to_rmats, how="outer", left_on=["groupID", "featureID"],
                               right_on=["GeneID", "DexseqFragment"])
rmatsID_col = dex_to_rmats_dexRes.pop("rMATS_ID")
dex_to_rmats_dexRes.insert(2, rmatsID_col.name, rmatsID_col)
dex_to_rmats_dexRes["groupID"].fillna(dex_to_rmats_dexRes["GeneID"], inplace=True)
dex_to_rmats_dexRes["featureID"].fillna(dex_to_rmats_dexRes["DexseqFragment"], inplace=True)
dex_to_rmats_dexRes.drop(columns=["GeneID", "DexseqFragment"], inplace=True)
dex_to_rmats_dexRes.sort_values(by=["groupID", "featureID"], inplace=True)
dex_to_rmats_dexRes.reset_index(drop=True, inplace=True)
del rmatsID_col

## Merge dex_to_rmats with rMATS results

In [4]:
dex_to_rmats_exploded = dex_to_rmats.copy()
dex_to_rmats_exploded["rMATS_ID"] = dex_to_rmats_exploded["rMATS_ID"].str.split(",")
dex_to_rmats_exploded = dex_to_rmats_exploded.explode("rMATS_ID")
#dex_to_rmats_exploded["rMATS_ID"] = dex_to_rmats_exploded["rMATS_ID"].astype(str)
dex_to_rmats_ex_A3 = dex_to_rmats_exploded.merge(A3SS_MATS, how="left", left_on=["GeneID", "rMATS_ID"],
                                                 right_on=["GeneID", "ID"])
dex_to_rmats_ex_A5 = dex_to_rmats_exploded.merge(A5SS_MATS, how="left", left_on=["GeneID", "rMATS_ID"],
                                                 right_on=["GeneID", "ID"])
dex_to_rmats_ex_SE = dex_to_rmats_exploded.merge(SE_MATS, how="left", left_on=["GeneID", "rMATS_ID"],
                                                 right_on=["GeneID", "ID"])
dex_to_rmats_ex_RI = dex_to_rmats_exploded.merge(RI_MATS, how="left", left_on=["GeneID", "rMATS_ID"],
                                                 right_on=["GeneID", "ID"])

dex_to_rmats_ex_MATS = pd.concat([dex_to_rmats_ex_A3, dex_to_rmats_ex_A5, dex_to_rmats_ex_SE, dex_to_rmats_ex_RI])
dex_to_rmats_ex_MATS.dropna(subset="ID.1", inplace=True)

del dex_to_rmats
del dex_to_rmats_ex_A3
del dex_to_rmats_ex_A5
del dex_to_rmats_ex_SE
del dex_to_rmats_ex_RI

## Merge the merged DEXSeq and rMATS dfs into one large exploded dataframe

In [5]:
dex_to_rmats_ex_dexRes = dex_to_rmats_dexRes.copy()
dex_to_rmats_ex_dexRes["rMATS_ID"] = dex_to_rmats_ex_dexRes["rMATS_ID"].str.split(",")
dex_to_rmats_ex_dexRes = dex_to_rmats_ex_dexRes.explode("rMATS_ID")
dex_to_rmats_ex_dexRes_MATS = dex_to_rmats_ex_dexRes.merge(dex_to_rmats_ex_MATS, how="left",
                                                           left_on=["groupID", "rMATS_ID", "featureID"],
                                                           right_on=["GeneID", "rMATS_ID", "DexseqFragment"])
dex_to_rmats_ex_dexRes_MATS[["padj", "FDR"]] = dex_to_rmats_ex_dexRes_MATS[["padj", "FDR"]].apply(pd.to_numeric)
del dex_to_rmats_ex_dexRes, dex_to_rmats_ex_MATS

# Get DEXSeq and rMATS counts for Detected, Tested, and Significant (DTS) Exons

### Total Exons Detected (by DEXSeq) comes from the combination of dex_to_rmats and dexRes

In [6]:
num_exons_detected = len(dex_to_rmats_dexRes)

### rMATS Detected Exons

In [7]:
exon_rmats_detected = dex_to_rmats_ex_dexRes_MATS.copy()
exon_rmats_detected = exon_rmats_detected.loc[exon_rmats_detected["rMATS_ID"].notna()]
exon_rmats_detected_dedup = exon_rmats_detected.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_rmats_detected_dedup = exon_rmats_detected_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_rmats_detected = len(exon_rmats_detected_dedup)
### ^^Now we have an rmats_detected df for all of the exons that are tested by rMATS

### rMATS Tested Exons

In [8]:
exon_rmats_tested = dex_to_rmats_ex_dexRes_MATS.copy()
exon_rmats_tested = exon_rmats_tested.dropna(subset="ID.1")
exon_rmats_tested_dedup = exon_rmats_tested.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_rmats_tested_dedup = exon_rmats_tested_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_rmats_tested = len(exon_rmats_tested_dedup)
### ^^Now we have an rmats_tested df for all of the exons that have tested events in rMATS

### rMATS Significant Exons

In [9]:
exon_rmats_sig = dex_to_rmats_ex_dexRes_MATS.copy()
exon_rmats_sig = exon_rmats_sig.loc[exon_rmats_sig["FDR"] <= .05]
exon_rmats_sig_dedup = exon_rmats_sig.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_rmats_sig_dedup = exon_rmats_sig_dedup.iloc[:, 19:]
num_exons_rmats_sig = len(exon_rmats_sig_dedup)
### ^^Now we have an rmats_sig df for all of the exons that have a significant event in rMATS

### DEXSeq Tested Exons

In [10]:
exon_dex_tested = dex_to_rmats_ex_dexRes_MATS.copy()
exon_dex_tested = exon_dex_tested.dropna(subset="padj")
exon_dex_tested_dedup = exon_dex_tested.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_tested_dedup = exon_dex_tested_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_tested = len(exon_dex_tested_dedup)

### Get DEXSeq Tested Exons Intersected with rMATS DTS

In [11]:
exon_dex_tested_rmats_detected = exon_dex_tested.copy()
exon_dex_tested_rmats_detected = exon_dex_tested_rmats_detected.loc[exon_dex_tested_rmats_detected["rMATS_ID"].notna()]
exon_dex_tested_rmats_detected_dedup = exon_dex_tested_rmats_detected.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_tested_rmats_detected_dedup = exon_dex_tested_rmats_detected_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_tested_rmats_detected = len(exon_dex_tested_rmats_detected_dedup)
## ^^Now we have the number of exons tested by DEXSeq and detected by rMATS

exon_dex_tested_rmats_tested = exon_dex_tested.copy()
exon_dex_tested_rmats_tested = exon_dex_tested_rmats_tested.loc[exon_dex_tested_rmats_tested["ID.1"].notna()]
exon_dex_tested_rmats_tested_dedup = exon_dex_tested_rmats_tested.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_tested_rmats_tested_dedup = exon_dex_tested_rmats_tested_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_tested_rmats_tested = len(exon_dex_tested_rmats_tested_dedup)
### ^^Now we have the number of exons tested by DEXSeq and tested by rMATS

exon_dex_tested_rmats_sig = exon_dex_tested.copy()
exon_dex_tested_rmats_sig = exon_dex_tested_rmats_sig.loc[exon_dex_tested_rmats_sig["FDR"] <= .05]
exon_dex_tested_rmats_sig_dedup = exon_dex_tested_rmats_sig.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_tested_rmats_sig_dedup = exon_dex_tested_rmats_sig_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_tested_rmats_sig = len(exon_dex_tested_rmats_sig_dedup)
### ^^Now we have the number of exons tested by DEXSeq and significant in rMATS

### DEXSeq Significant Exons

In [12]:
exon_dex_sig = dex_to_rmats_ex_dexRes_MATS.copy()
exon_dex_sig = exon_dex_sig.loc[exon_dex_sig["padj"] <= .05]
exon_dex_sig_dedup = exon_dex_sig.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_sig_dedup = exon_dex_sig_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_sig = len(exon_dex_sig_dedup)

### Get DEXSeq Significant Exons Intersected with rMATS DTS

In [13]:
exon_dex_sig_rmats_detected = exon_dex_sig.copy()
exon_dex_sig_rmats_detected = exon_dex_sig_rmats_detected.loc[exon_dex_sig_rmats_detected["rMATS_ID"].notna()]
exon_dex_sig_rmats_detected_dedup = exon_dex_sig_rmats_detected.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_sig_rmats_detected_dedup = exon_dex_sig_rmats_detected_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_sig_rmats_detected = len(exon_dex_sig_rmats_detected_dedup)
## ^^Now we have the number of exons significant in DEXSeq and detected by rMATS

exon_dex_sig_rmats_tested = exon_dex_sig.copy()
exon_dex_sig_rmats_tested = exon_dex_sig_rmats_tested.loc[exon_dex_sig_rmats_tested["ID.1"].notna()]
exon_dex_sig_rmats_tested_dedup = exon_dex_sig_rmats_tested.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_sig_rmats_tested_dedup = exon_dex_sig_rmats_tested_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_sig_rmats_tested = len(exon_dex_sig_rmats_tested_dedup)
### ^^Now we have the number of exons significant in DEXSeq and tested by rMATS

exon_dex_sig_rmats_sig = exon_dex_sig.copy()
exon_dex_sig_rmats_sig = exon_dex_sig_rmats_sig.loc[exon_dex_sig_rmats_sig["FDR"] <= .05]
exon_dex_sig_rmats_sig_dedup = exon_dex_sig_rmats_sig.drop_duplicates(subset=["groupID", "featureID"], keep="first")
exon_dex_sig_rmats_sig_dedup = exon_dex_sig_rmats_sig_dedup[["groupID", "featureID", "padj", "rMATS_ID"]]
num_exons_dex_sig_rmats_sig = len(exon_dex_sig_rmats_sig_dedup)
### ^^Now we have the number of exons significant in DEXSeq and significant in rMATS

---
# Counting Events

## Get rmats_to_dex

In [14]:
A3SS_to_dex = pd.read_table(
    "/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.fromGTF.A3SS.txt",
	dtype=str)
A3SS_to_dex = A3SS_to_dex[A3SS_to_dex["GeneID"] != "GeneID"]
A3SS_to_dex["ID"] = "A3SS_" + A3SS_to_dex["ID"].astype(str)

A5SS_to_dex = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.fromGTF.A5SS.txt",
	dtype=str)
A5SS_to_dex = A5SS_to_dex[A5SS_to_dex["GeneID"] != "GeneID"]
A5SS_to_dex["ID"] = "A5SS_" + A5SS_to_dex["ID"].astype(str)

SE_to_dex = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.fromGTF.SE.txt",
	dtype=str)
SE_to_dex = SE_to_dex[SE_to_dex["GeneID"] != "GeneID"]
SE_to_dex["ID"] = "SE_" + SE_to_dex["ID"].astype(str)

RI_to_dex = pd.read_table(
	"/home/dwito/merging_rmats_dexseq/genome_wide_analysis_bonnal/Bnaive-CD8naive/grase_results/results/tmp/combined.fromGTF.RI.txt",
	dtype=str)
RI_to_dex = RI_to_dex[RI_to_dex["GeneID"] != "GeneID"]
RI_to_dex["ID"] = "RI_" + RI_to_dex["ID"].astype(str)

rmats_to_dex = pd.concat([A3SS_to_dex, A5SS_to_dex, SE_to_dex, RI_to_dex])
del A3SS_to_dex, A5SS_to_dex, RI_to_dex, SE_to_dex
rmats_to_dex.sort_values(by=["GeneID", "ID"], inplace=True)
rmats_to_dex.reset_index(drop=True, inplace=True)

## Merge rmats_to_dex with rMATS results

In [27]:
rmats_to_dex_A3 = rmats_to_dex.merge(A3SS_MATS, how="left", on=["GeneID", "ID"])
#del A3SS_MATS
rmats_to_dex_A5 = rmats_to_dex.merge(A5SS_MATS, how="left", on=["GeneID", "ID"])
#del A5SS_MATS
rmats_to_dex_SE = rmats_to_dex.merge(SE_MATS, how="left", on=["GeneID", "ID"])
#del SE_MATS
rmats_to_dex_RI = rmats_to_dex.merge(RI_MATS, how="left", on=["GeneID", "ID"])
#del RI_MATS

rmats_to_dex_MATS = pd.concat([rmats_to_dex_A3, rmats_to_dex_A5, rmats_to_dex_SE, rmats_to_dex_RI])
rmats_to_dex_MATS.dropna(subset="ID.1", inplace=True)

del rmats_to_dex_A3
del rmats_to_dex_A5
del rmats_to_dex_SE
del rmats_to_dex_RI

NameError: name 'rmats_to_dex' is not defined

## Merge rmats_to_dex with DEXSeq results

In [16]:
rmats_to_dex_exploded = rmats_to_dex.copy()
rmats_to_dex_exploded["DexseqFragment"] = rmats_to_dex_exploded["DexseqFragment"].str.split(",")
rmats_to_dex_exploded = rmats_to_dex_exploded.explode("DexseqFragment")
### ^^Now we have all rmats detected events mapped to each relevant dexseq exon (separated by rows - exploded)
rmats_to_dex_ex_dexRes = rmats_to_dex_exploded.merge(
	dexseqResults.rename(columns={"groupID": "GeneID", "featureID": "DexseqFragment"}), how="left",
	on=["GeneID", "DexseqFragment"])
del rmats_to_dex_exploded
#rmats_to_dex_ex_dexRes[["padj"]] = rmats_to_dex_ex_dexRes[["padj"]].apply(pd.to_numeric)

## Merge the merged rMATS and DEXSeq dfs into one large exploded dataframe

In [17]:
rmats_to_dex_ex_MATS = rmats_to_dex_MATS.copy()
rmats_to_dex_ex_MATS["DexseqFragment"] = rmats_to_dex_ex_MATS["DexseqFragment"].str.split(",")
rmats_to_dex_ex_MATS = rmats_to_dex_ex_MATS.explode("DexseqFragment")
### ^^Now we have all tested events (that have read counts) mapped to each exon in those events (exploded), merged with rMATS results
rmats_to_dex_ex_MATS_dexRes = rmats_to_dex_ex_MATS.merge(rmats_to_dex_ex_dexRes, how="outer",
                                                         on=["GeneID", "ID", "DexseqFragment"])
rmats_to_dex_ex_MATS_dexRes[["padj", "FDR"]] = rmats_to_dex_ex_MATS_dexRes[["padj", "FDR"]].apply(pd.to_numeric)
### ^^Now we have all tested events (that have read counts) mapped to each exon in those events (exploded), merged with rMATS results and DEXSeq results

# Get rMATS and DEXSeq counts for DTS Events

### Total Events Detected (by rMATS and by DEXSeq) comes from rmats_to_dex

In [18]:
num_events_detected = len(rmats_to_dex)
del rmats_to_dex, rmats_to_dex_ex_dexRes, rmats_to_dex_ex_MATS

### DEXSeq Tested Events

In [19]:
event_dex_tested = rmats_to_dex_ex_MATS_dexRes.copy()
event_dex_tested = event_dex_tested.loc[event_dex_tested["padj"].notna()]
event_dex_tested_dedup = event_dex_tested.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_dex_tested_dedup = event_dex_tested_dedup[["GeneID", "ID", "DexseqFragment","padj"]]
num_events_dex_tested = len(event_dex_tested_dedup)

### DEXSeq Significant Events

In [20]:
event_dex_sig = rmats_to_dex_ex_MATS_dexRes.copy()
event_dex_sig = event_dex_sig.loc[event_dex_sig["padj"] <= .05]
event_dex_sig_dedup = event_dex_sig.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_dex_sig_dedup = event_dex_sig_dedup[["GeneID", "ID", "DexseqFragment","padj"]]
num_events_dex_sig = len(event_dex_sig_dedup)

### rMATS Tested Events

In [21]:
event_rmats_tested = rmats_to_dex_ex_MATS_dexRes.copy()
event_rmats_tested = event_rmats_tested.dropna(subset="ID.1")
event_rmats_tested_dedup = event_rmats_tested.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_rmats_tested_dedup = event_rmats_tested_dedup[["GeneID", "ID", "FDR", "DexseqFragment","padj"]]
num_events_rmats_tested = len(event_rmats_tested_dedup)

### Get rMATS Tested Events Intersected with DEXSeq TS (D is just rmats_tested)

In [22]:
event_rmats_tested_dex_tested = event_rmats_tested.copy()
event_rmats_tested_dex_tested = event_rmats_tested_dex_tested.loc[event_rmats_tested_dex_tested["padj"].notna()]
event_rmats_tested_dex_tested_dedup = event_rmats_tested_dex_tested.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_rmats_tested_dex_tested_dedup = event_rmats_tested_dex_tested_dedup[["GeneID", "ID", "FDR", "DexseqFragment","padj"]]
num_events_rmats_tested_dex_tested = len(event_rmats_tested_dex_tested_dedup)

event_rmats_tested_dex_sig = event_rmats_tested.copy()
event_rmats_tested_dex_sig = event_rmats_tested_dex_sig.loc[event_rmats_tested_dex_sig["padj"] <= .05]
event_rmats_tested_dex_sig_dedup = event_rmats_tested_dex_sig.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_rmats_tested_dex_sig_dedup = event_rmats_tested_dex_sig_dedup[["GeneID", "ID", "FDR", "DexseqFragment","padj"]]
num_events_rmats_tested_dex_sig = len(event_rmats_tested_dex_sig_dedup)

### rMATS Significant Events

In [23]:
event_rmats_sig = rmats_to_dex_ex_MATS_dexRes.copy()
event_rmats_sig = event_rmats_sig.loc[event_rmats_sig["FDR"] <= .05]
event_rmats_sig_dedup = event_rmats_sig.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_rmats_sig_dedup = event_rmats_sig_dedup[["GeneID", "ID", "FDR", "DexseqFragment","padj"]]
num_events_rmats_sig = len(event_rmats_sig_dedup)

### Get rMATS Significant Events Intersected with DEXSeq TS (D is just rmats_sig)

In [24]:
event_rmats_sig_dex_tested = event_rmats_sig.copy()
event_rmats_sig_dex_tested = event_rmats_sig_dex_tested.loc[event_rmats_sig_dex_tested["padj"].notna()]
event_rmats_sig_dex_tested_dedup = event_rmats_sig_dex_tested.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_rmats_sig_dex_tested_dedup = event_rmats_sig_dex_tested_dedup[["GeneID", "ID", "FDR", "DexseqFragment","padj"]]
num_events_rmats_sig_dex_tested = len(event_rmats_sig_dex_tested_dedup)

event_rmats_sig_dex_sig = event_rmats_sig.copy()
event_rmats_sig_dex_sig = event_rmats_sig_dex_sig.loc[event_rmats_sig_dex_sig["padj"] <= .05]
event_rmats_sig_dex_sig_dedup = event_rmats_sig_dex_sig.drop_duplicates(subset=["GeneID", "ID"], keep="first")
event_rmats_sig_dex_sig_dedup = event_rmats_sig_dex_sig_dedup[["GeneID", "ID", "FDR", "DexseqFragment","padj"]]
num_events_rmats_sig_dex_sig = len(event_rmats_sig_dex_sig_dedup)

In [25]:

'''#A3SS_to_dex = A3SS_to_dex.sort_values(by=["GeneID", "ID"])
### ^^Now we have all rmats detected events mapped to dexseq exons (in a comma separated list)

events_rmats_detected_dex_sig = rmats_to_dex_ex_dexRes.loc[rmats_to_dex_ex_dexRes["padj"] <= .05]
events_rmats_detected_dex_sig_dedup = events_rmats_detected_dex_sig.drop_duplicates(subset=["GeneID", "ID"],
                                                                                    keep="first")
num_events_rmats_detected_dex_sig = len(events_rmats_detected_dex_sig_dedup)
### ^^Now we have all rmats detected events that have at least one dexseq significant exon


rmats_to_dex_MATS[["FDR"]] = rmats_to_dex_MATS[["FDR"]].apply(pd.to_numeric)

### ^^Now we have all rmats tested events
events_rmats_sig = rmats_to_dex_MATS.loc[rmats_to_dex_MATS["FDR"] <= .05]
num_events_rmats_sig = len(events_rmats_sig)
### ^^Now we have all rmats significant events

events_rmats_tested_dex_sig = rmats_to_dex_ex_MATS_dexRes.loc[
	((rmats_to_dex_ex_MATS_dexRes["ID.1"].notna()) & (rmats_to_dex_ex_MATS_dexRes["padj"] <= .05))]
events_rmats_tested_dex_sig_dedup = events_rmats_tested_dex_sig.drop_duplicates(subset=["GeneID", "ID"])
num_events_rmats_tested_dex_sig = len(events_rmats_tested_dex_sig_dedup)
### ^^Now we have all tested events with at least one significant exon in dexseq
events_rmats_sig_dex_sig = rmats_to_dex_ex_MATS_dexRes[["GeneID", "ID", "FDR", "DexseqFragment", "padj"]].loc[
	((rmats_to_dex_ex_MATS_dexRes["FDR"] <= .05) & (rmats_to_dex_ex_MATS_dexRes["padj"] <= .05))].drop_duplicates(
	subset=["GeneID", "ID"])
num_events_rmats_sig_dex_sig = len(events_rmats_sig_dex_sig)
### ^^Now we have all events that are significant in both rmats and dexseq'''

'#A3SS_to_dex = A3SS_to_dex.sort_values(by=["GeneID", "ID"])\n### ^^Now we have all rmats detected events mapped to dexseq exons (in a comma separated list)\n\nevents_rmats_detected_dex_sig = rmats_to_dex_ex_dexRes.loc[rmats_to_dex_ex_dexRes["padj"] <= .05]\nevents_rmats_detected_dex_sig_dedup = events_rmats_detected_dex_sig.drop_duplicates(subset=["GeneID", "ID"],\n                                                                                    keep="first")\nnum_events_rmats_detected_dex_sig = len(events_rmats_detected_dex_sig_dedup)\n### ^^Now we have all rmats detected events that have at least one dexseq significant exon\n\n\nrmats_to_dex_MATS[["FDR"]] = rmats_to_dex_MATS[["FDR"]].apply(pd.to_numeric)\n\n### ^^Now we have all rmats tested events\nevents_rmats_sig = rmats_to_dex_MATS.loc[rmats_to_dex_MATS["FDR"] <= .05]\nnum_events_rmats_sig = len(events_rmats_sig)\n### ^^Now we have all rmats significant events\n\nevents_rmats_tested_dex_sig = rmats_to_dex_ex_MATS_dexRes.loc[\n

---
# Summary Table

In [26]:
data = [["", ""],
        ["Total Exons Detected", num_exons_detected],
        ["DEXSeq Tested Exons", num_exons_dex_tested],
        ["DEXSeq Sig Exons", num_exons_dex_sig],
        ["rMATS Detected Exons", num_exons_rmats_detected],
        ["rMATS Tested Exons", num_exons_rmats_tested],
        ["rMATS Sig Exons", num_exons_rmats_sig],
        ["", ""],
        ["DEXSeq Tested & rMATS Detected Exons", num_exons_dex_tested_rmats_detected],
        ["DEXSeq Tested & rMATS Tested Exons", num_exons_dex_tested_rmats_tested],
        ["DEXSeq Tested & rMATS Sig Exons", num_exons_dex_tested_rmats_sig],
        ["DEXSeq Sig & rMATS Detected Exons", num_exons_dex_sig_rmats_detected],
        ["DEXSeq Sig & rMATS Tested Exons", num_exons_dex_sig_rmats_tested],
        ["DEXSeq Sig & rMATS Sig Exons", num_exons_dex_sig_rmats_sig],
        ["", ""],
        ["Total Events Detected", num_events_detected],
        ["rMATS Tested Events", num_events_rmats_tested],
        ["rMATS Sig Events", num_events_rmats_sig],
        ["DEXSeq Tested Events", num_events_dex_tested],
        ["DEXSeq Sig Events", num_events_dex_sig],
        ["", ""],
        ["rMATS Tested & DEXSeq Tested Events", num_events_rmats_tested_dex_tested],
        ["rMATS Tested & DEXSeq Sig Events", num_events_rmats_tested_dex_sig],
        ["rMATS Sig & DEXSeq Tested Events", num_events_rmats_sig_dex_tested],
        ["rMATS Sig & DEXSeq Sig Events", num_events_rmats_sig_dex_sig]]

summary_table = pd.DataFrame(data, columns=["CountType", "Counts"])